# This code is written to split the raw signals from the MIT BIH sleep dataset into 30 second epochs. These epochs are then split further into 4 channels (ECG, BP, EEG and Resp)

In [1]:
from IPython.display import display
import matplotlib.pyplot as plt
import numpy as np
import os
import shutil
import posixpath
import wfdb
import csv
from scipy import signal
import pandas as pd

Each individual slice/piece can be accessed by using obs1_split[i] where i = 1:240
This pertains to subject 1. Similar sizes are calculated for each subject

In [2]:
fs = 250
obs = wfdb.rdsamp('mit-bih-polysomnographic-database-1.0.0/slp67x')
obs = obs[0]
size = len(obs)/(fs*30)
obs_split = np.split(obs,size)
# 240 is derived by dividing the entire record by number of samples in a 30 second period (1800000/(250*30)) - 
# here Fs: 250 and each annotation duration is 30 seconds

## Set this value based on the subject being used
## For example: if I want to extract 5th subject's information then set sub = 5
##(Since python's indexing starts from 0, I added the -1)
sub = 18 - 1;

In [3]:
np.shape(obs_split[0])

(7500, 7)

In [4]:
apnea_indicators = pd.read_excel('Annotations - No apnea recode\processed_apnea_indicators recoded.xlsx')
sleep_stages = pd.read_excel('Annotations - No apnea recode\processed_sleep_stages recoded.xlsx')
HA_indicators = pd.read_excel('Annotations - No apnea recode\processed_HA_indicators recoded.xlsx')
LA_indicators = pd.read_excel('Annotations - No apnea recode\processed_LA_indicators recoded.xlsx')

In [5]:
# Splitting the apnea indicator array of observation 1 into individual annotations
size = len(obs)/(fs*30)
size = int(size)
aei = ["" for i in range(size)]
for i in range (size):
    individual_aei = apnea_indicators.iloc[:,sub]
    aei[i] = individual_aei[i]

In [6]:
size

154

In [7]:
np.shape(apnea_indicators)

(780, 18)

In [8]:
ss = ["" for i in range(size)]
for i in range(size):
    individual_ss = sleep_stages.iloc[:,sub]
    ss[i] = individual_ss[i]

In [9]:
hei = ["" for i in range(size)]
for i in range(size):
    individual_hei = HA_indicators.iloc[:,sub]
    hei[i] = individual_hei[i]

In [10]:
lei = ["" for i in range(size)]
for i in range(size):
    individual_lei = LA_indicators.iloc[:,sub]
    lei[i] = individual_lei[i]

# Code to make a dataframe of ECG, Bp, EEG and resp amplitude data with their corresponding annotations

In [11]:
ECG = {
    'SS': [ss[i] for i in range(size)],
    'Apnea_Indicator' : [aei[i] for i in range (size)],
    'HEI': [hei[i] for i in range(size)],
    'LEI': [lei[i] for i in range(size)],
    'ECG_amps': [obs_split[i][:,0] for i in range (size)] #obs_split[i][:,0] corresponds to ECG data.
    # if we change [:,0] to [:,1] then it corresponds to BP. [:,2] for EEG and [:,3] for Resp
}
ECG = pd.DataFrame(ECG, columns = ['SS','Apnea_Indicator','HEI','LEI','ECG_amps'])
print(ECG)

     SS Apnea_Indicator   HEI   LEI  \
0     1               X  NoHA  NoLA   
1     1             NoA  NoHA  NoLA   
2    W                X  NoHA  NoLA   
3     1             NoA  NoHA  NoLA   
4     1               X  NoHA  NoLA   
..   ..             ...   ...   ...   
149   W             NoA  NoHA  NoLA   
150   W             NoA  NoHA  NoLA   
151   W             NoA  NoHA  NoLA   
152   W             NoA  NoHA  NoLA   
153   W             NoA  NoHA  NoLA   

                                              ECG_amps  
0    [-0.032, -0.03, -0.038, -0.032, -0.028, -0.032...  
1    [-0.04, -0.036, -0.04, -0.038, -0.034, -0.032,...  
2    [-0.04, -0.044, -0.044, -0.04, -0.04, -0.042, ...  
3    [0.072, 0.042, 0.03, 0.002, -0.008, -0.02, -0....  
4    [-0.064, -0.062, -0.05, -0.052, -0.046, -0.054...  
..                                                 ...  
149  [-0.016, -0.012, -0.012, -0.008, -0.012, -0.00...  
150  [0.0, 0.014, 0.016, 0.024, 0.024, 0.03, 0.044,...  
151  [-0.032, -0.0

In [12]:
BP = {
    'SS': [ss[i] for i in range(size)],
    'Apnea_Indicator' : [aei[i] for i in range (size)],
    'HEI': [hei[i] for i in range(size)],
    'LEI': [lei[i] for i in range(size)],
    'BP_amps': [obs_split[i][:,1] for i in range (size)] #obs1_split[i][:,0] corresponds to ECG data.
    # if we change [:,0] to [:,1] then it corresponds to BP. [:,2] for EEG and [:,3] for Resp
}
BP = pd.DataFrame(BP, columns = ['SS','Apnea_Indicator','HEI','LEI','BP_amps'])
print(BP)

     SS Apnea_Indicator   HEI   LEI  \
0     1               X  NoHA  NoLA   
1     1             NoA  NoHA  NoLA   
2    W                X  NoHA  NoLA   
3     1             NoA  NoHA  NoLA   
4     1               X  NoHA  NoLA   
..   ..             ...   ...   ...   
149   W             NoA  NoHA  NoLA   
150   W             NoA  NoHA  NoLA   
151   W             NoA  NoHA  NoLA   
152   W             NoA  NoHA  NoLA   
153   W             NoA  NoHA  NoLA   

                                               BP_amps  
0    [80.09243624629136, 80.48060637349471, 80.4806...  
1    [87.72644874795725, 86.30315828154497, 85.1386...  
2    [81.77450679750588, 81.38633667030253, 80.8687...  
3    [119.81517926343425, 119.16822905142865, 118.3...  
4    [119.81517926343425, 117.61554854261526, 115.1...  
..                                                 ...  
149  [55.24954810527691, 55.24954810527691, 54.7319...  
150  [77.50463539826902, 77.11646527106566, 76.5989...  
151  [52.402967172

In [13]:
EEG = {
    'SS': [ss[i] for i in range(size)],
    'Apnea_Indicator' : [aei[i] for i in range (size)],
    'HEI': [hei[i] for i in range(size)],
    'LEI': [lei[i] for i in range(size)],
    'EEG_amps': [obs_split[i][:,2] for i in range (size)] #obs1_split[i][:,0] corresponds to ECG data.
    # if we change [:,0] to [:,1] then it corresponds to BP. [:,2] for EEG and [:,3] for Resp
}
EEG = pd.DataFrame(EEG, columns = ['SS','Apnea_Indicator','HEI','LEI','EEG_amps'])
print(EEG)

     SS Apnea_Indicator   HEI   LEI  \
0     1               X  NoHA  NoLA   
1     1             NoA  NoHA  NoLA   
2    W                X  NoHA  NoLA   
3     1             NoA  NoHA  NoLA   
4     1               X  NoHA  NoLA   
..   ..             ...   ...   ...   
149   W             NoA  NoHA  NoLA   
150   W             NoA  NoHA  NoLA   
151   W             NoA  NoHA  NoLA   
152   W             NoA  NoHA  NoLA   
153   W             NoA  NoHA  NoLA   

                                              EEG_amps  
0    [-0.04303115204130991, -0.040727781372432006, ...  
1    [-0.011516853344389514, -0.018636362684557577,...  
2    [-0.02805924269360354, -0.02764044802653483, -...  
3    [-0.006910112006633708, -0.007328906673702417,...  
4    [-0.006386618672797821, -0.005967824005729112,...  
..                                                 ...  
149  [-0.01842696535102322, -0.019159856018393463, ...  
150  [-0.018322266684256044, -0.018322266684256044,...  
151  [-0.010574565

In [14]:
Resp = {
    'SS': [ss[i] for i in range(size)],
    'Apnea_Indicator' : [aei[i] for i in range (size)],
    'HEI': [hei[i] for i in range(size)],
    'LEI': [lei[i] for i in range(size)],
    'Resp_amps': [obs_split[i][:,3] for i in range (size)] #obs1_split[i][:,0] corresponds to ECG data.
    # if we change [:,0] to [:,1] then it corresponds to BP. [:,2] for EEG and [:,3] for Resp
}
Resp = pd.DataFrame(Resp, columns = ['SS','Apnea_Indicator','HEI','LEI','Resp_amps'])
print(Resp)

     SS Apnea_Indicator   HEI   LEI  \
0     1               X  NoHA  NoLA   
1     1             NoA  NoHA  NoLA   
2    W                X  NoHA  NoLA   
3     1             NoA  NoHA  NoLA   
4     1               X  NoHA  NoLA   
..   ..             ...   ...   ...   
149   W             NoA  NoHA  NoLA   
150   W             NoA  NoHA  NoLA   
151   W             NoA  NoHA  NoLA   
152   W             NoA  NoHA  NoLA   
153   W             NoA  NoHA  NoLA   

                                             Resp_amps  
0    [0.15778120717753197, 0.15778120717753197, 0.1...  
1    [0.18198873485408482, 0.18458239853371547, 0.1...  
2    [0.15345843437814752, 0.15345843437814752, 0.1...  
3    [0.13616734318060977, 0.1353027886207329, 0.13...  
4    [0.09380416974664228, 0.0899136742271963, 0.08...  
..                                                 ...  
149  [0.065273869270705, 0.0661384238305819, 0.0657...  
150  [0.06095109647132057, 0.06051881919138212, 0.0...  
151  [0.0631124828

# Extracting sub-band EEG power

In [15]:
# Sort the strings and remove whitespace
#sort_var1 = ''.join(sorted(var1)).strip()
#sort_var2 = ''.join(sorted(var2)).strip()

In [16]:
import mne
from mne.datasets.sleep_physionet.age import fetch_data
from mne.time_frequency import psd_welch

from sklearn.ensemble import RandomForestClassifier
from sklearn.metrics import accuracy_score
from sklearn.metrics import confusion_matrix
from sklearn.metrics import classification_report
from sklearn.pipeline import make_pipeline
from sklearn.preprocessing import FunctionTransformer

In [17]:
# specific frequency bands - referred from Danielle and Faes (2014)
bands = {"delta": [0.5, 3],
                  "theta": [3, 8],
                  "alpha": [8, 12],
                  "sigma": [12, 16],
                  "beta": [16, 25]}

In [18]:
PEEG = np.zeros([size,5],float)
from scipy.signal import welch
for j in range(size):
    temp = EEG.EEG_amps[j]
    f, psd = welch(temp, fs=250,nperseg=7500,noverlap=1875) #30% overelap - similar to Danielle and Faes (2014)
    psd /= np.sum(psd, axis=-1, keepdims=True) #Normalizing the PSD
    i = 0
    psds_band = np.zeros(5,float) #Initiating an empty array to contain the power for each band
    for fmin, fmax in bands.values():
            psds_band[i] = psd[(f >= fmin) & (f < fmax)].mean(axis=-1)
            PEEG[j,i] = psds_band[i]
            i = i+1

In [19]:
PEEGd = PEEG[:,0]
PEEGt = PEEG[:,1]
PEEGa = PEEG[:,2]
PEEGs = PEEG[:,3]
PEEGb = PEEG[:,4]

# ECG Feature extraction - incomplete

In [20]:
#from ecgdetectors import Detectors
#detectors = Detectors(fs)

In [21]:
#r_peaks = detectors.pan_tompkins_detector(ECG1.ECG_amps[0]) #https://github.com/berndporr/py-ecg-detectors

In [22]:
# specific frequency bands - referred from Danielle and Faes (2014)
ECG_bands = {"HF": [0.15, 0.4],
                  "LF": [0.04,0.4]
        }

In [23]:
PECG = np.zeros([size,2],float)
from scipy.signal import welch
for j in range(size):
    temp = ECG.ECG_amps[j]
    f, psd = welch(temp, fs=250,nperseg=7500,noverlap=1875) #50% overelap - similar to Danielle and Faes (2014)
    psd /= np.sum(psd, axis=-1, keepdims=True) #Normalizing the PSD
    i = 0
    ECG_psds_band = np.zeros(2,float) #Initiating an empty array to contain the power for each band
    for fmin, fmax in ECG_bands.values():
            ECG_psds_band[i] = psd[(f >= fmin) & (f < fmax)].mean(axis=-1)
            PECG[j,i] = ECG_psds_band[i]
            i = i+1

In [24]:
PECG_HF = PECG[:,0]
PECG_LF = PECG[:,1]

In [25]:
np.shape(PECG_HF)

(154,)

# Breathing rate and BPM detection using ECG
### additional measurements can also be computed using this package (heartpy) , if needed

In [26]:
import heartpy as hp
BR = np.zeros(size,'float')
BPM = np.zeros(size,'float')
for j in range(size):
    data = ECG.ECG_amps[j]
    fs = 250
    working_data, measures = hp.process(data, fs)
    BR[j] = measures['breathingrate']
    BPM[j] = measures['bpm']

In [27]:
measures

{'bpm': 83.90177353342429,
 'ibi': 715.1219512195122,
 'sdnn': 27.294415690760985,
 'sdsd': 7.8255990186055415,
 'rmssd': 12.231107881136523,
 'pnn20': 0.1,
 'pnn50': 0.0,
 'hr_mad': 20.0,
 'sd1': 8.644073113989723,
 'sd2': 38.0525951808809,
 's': 1033.3622334641066,
 'sd1/sd2': 0.22716119814957694,
 'breathingrate': 0.1364256480218281}

## Computing AUC respiration

In [28]:
# Using the trapezoidal rule np.trapz
#x = [i+1 for i in range(7500)] - checked it, the AUC with or without x is the same
AUC = np.zeros(size,'float')
for i in range(size):
    AUC[i] = np.trapz(Resp['Resp_amps'][i])

## Splitting each of the annotation string into corresponding 30 second bits

In [29]:
ss_split = [0 for i in range(size)]
for i in range (size):
    temp = str(ss[i])
    ss_split[i] = str.split(temp)

In [30]:
aei_split = [0 for i in range(size)]
for i in range (size):
    temp = str(aei[i])
    aei_split[i] = str.split(temp)

In [31]:
hei_split = [0 for i in range(size)]
for i in range (size):
    temp = str(hei[i])
    hei_split[i] = str.split(temp)

In [32]:
lei_split = [0 for i in range(size)]
for i in range (size):
    temp = str(lei[i])
    lei_split[i] = str.split(temp)

In [33]:
SS = pd.DataFrame(columns=['SS'], data=ss_split)
AEI = pd.DataFrame(columns=['AEI'], data=aei_split)
HEI = pd.DataFrame(columns=['HEI'], data=hei_split)
LEI = pd.DataFrame(columns=['LEI'], data=lei_split)
AUC = pd.DataFrame(columns=['AUC'], data = AUC)

## Integrating all the extracted features into a pandas df for easy access

In [34]:
### Change Subj1 to appropriate value

In [35]:
Subj = {
    'SS' : SS['SS'],
    'AEI' : AEI['AEI'],
    'HEI' : HEI['HEI'],
    'LEI' : LEI['LEI'],
    'PEEGd': PEEGd, 
    'PEEGt': PEEGt,
    'PEEGa': PEEGa,
    'PEEGs': PEEGs,
    'PEEGb': PEEGb,
    'PECG_HF': PECG_HF,
    'PECG_LF': PECG_LF,
    'BR': BR,
    'BPM': BPM,
    'AUC_Resp':AUC['AUC']
}
Subj = pd.DataFrame(Subj, columns = ['SS','AEI','HEI','LEI','PEEGd','PEEGt','PEEGa','PEEGs','PEEGb','PECG_HF','PECG_LF','BR','BPM','AUC_Resp'])
print(Subj)
%store Subj

    SS  AEI   HEI   LEI     PEEGd     PEEGt     PEEGa     PEEGs     PEEGb  \
0    1    X  NoHA  NoLA  0.005613  0.000849  0.000446  0.000187  0.000084   
1    1  NoA  NoHA  NoLA  0.002448  0.000810  0.000427  0.000175  0.000099   
2    W    X  NoHA  NoLA  0.006164  0.000409  0.000234  0.000143  0.000069   
3    1  NoA  NoHA  NoLA  0.002447  0.000919  0.000361  0.000230  0.000128   
4    1    X  NoHA  NoLA  0.002632  0.001171  0.000388  0.000268  0.000152   
..  ..  ...   ...   ...       ...       ...       ...       ...       ...   
149  W  NoA  NoHA  NoLA  0.001806  0.000759  0.001240  0.000318  0.000149   
150  W  NoA  NoHA  NoLA  0.001120  0.000763  0.002304  0.000345  0.000165   
151  W  NoA  NoHA  NoLA  0.003564  0.000232  0.000260  0.000090  0.000054   
152  W  NoA  NoHA  NoLA  0.004009  0.000246  0.000087  0.000072  0.000050   
153  W  NoA  NoHA  NoLA  0.005996  0.000334  0.000115  0.000074  0.000064   

      PECG_HF   PECG_LF        BR        BPM     AUC_Resp  
0    0.000067  

In [36]:
from rpy2.robjects import pandas2ri
pandas2ri.activate()

from rpy2.robjects.packages import importr

base = importr('base')
# call an R function on a Pandas DataFrame
base.summary(Subj)

from rpy2.robjects import pandas2ri
import rpy2.robjects as robjects
## get a reference to the R function 
write_csv = robjects.r('write.csv')
## save 
write_csv(Subj,'ExportedS18recoded.csv')

<rpy2.rinterface_lib.sexp.NULLType object at 0x000001FE4B71BA40> [RTYPES.NILSXP]